In [1]:
import requests
import pandas as pd

date = pd.to_datetime('2023/9/1')

url = 'https://www.twse.com.tw/pcversion/zh/exchangeReport/MI_INDEX'

payloads = {
    'response': 'html',
    'date': date.strftime('%Y%m%d'),
    'type': 'ALLBUT0999'
}

headers = {
    'Users-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'
}

response = requests.get(url, params=payloads, headers=headers)

response.text

'<!doctype html>\n<html lang="zh">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <title>TWSE 臺灣證券交易所 - 報表</title>\n  <style>\n    table { border-collapse: collapse; margin: 1em 0; }\n    table, th, td { border: 1px solid lightgray; }\n    table thead th, table thead td { text-align: center; background-color: #eee; }\n    table thead div { font-size: 1.5em; padding: 10px; }\n    table tbody tr:nth-child(even){ background-color:#f8f8f8; }\n    \n    td { padding: 4px; }\n    td>p { margin: 0; text-align: center; }\n    div.notes { margin: 2em 0 4em 0; line-height: 1.5em; }\n\n    table tbody td { text-align: right; }\n    table tbody td:first-child { text-align: left; }\n\n  </style>\n</head>\n<body>\n<div>\n    \n        <h2></h2>\n        \n            <table>\n            <thead>\n                <tr>\n                    \n                        <th scope="col" colspan=\'6\'>112年09月01日 價格指數(

2-1 
資料整理

In [2]:
df = pd.read_html(response.text)[-1]
#[-1]最後一張表格
df.columns = df.columns.get_level_values(2)
df['漲跌價差'] = df['漲跌價差'].where(df['漲跌(+/-)'] != '-', -df['漲跌價差'])
df = df.drop(['證券名稱','漲跌(+/-)'],axis = 1)
#將漲跌方向與漲跌價差合併
df['日期'] = pd.to_datetime(date)
df = df.set_index(['證券代號','日期'])
df = df.apply(pd.to_numeric, errors = 'coerce')
#errors = 'coerce' : 將資料數字化，若無法數字化則呈現NaN
df.drop(df[df['收盤價'].isnull()].index, inplace=True)
df


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,日期,,,,,,,,,,,,,
0050,2023-09-01,3298119,5596,415112581,125.50,126.30,125.10,125.90,0.20,125.85,12,125.90,38,0.00
0051,2023-09-01,57694,223,4084572,71.25,71.25,70.55,70.55,-0.35,70.50,1,70.55,47,0.00
0052,2023-09-01,89203,222,10430606,116.80,117.30,116.65,116.85,0.00,116.65,6,116.75,21,0.00
0053,2023-09-01,9231,1010,625518,67.90,68.00,67.50,67.50,-0.55,67.25,1,67.75,10,0.00
0055,2023-09-01,123518,195,2894686,23.32,23.49,23.30,23.45,0.10,23.45,1,23.46,2,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2023-09-01,70188,111,1373176,19.50,19.70,19.50,19.65,0.05,19.60,2,19.65,6,5.31
9945,2023-09-01,4446023,3364,162990865,36.85,36.90,36.50,36.60,-0.25,36.55,154,36.60,15,26.52
9946,2023-09-01,253303,129,3621322,14.35,14.40,14.20,14.20,-0.20,14.20,10,14.25,10,33.02


2-2 
依股價震幅找出活躍股

In [5]:
df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
df['股價震幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100


In [6]:
#找出震幅高達6% 
df[(df['股價震幅']>6)]

,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價震幅
證券代號,日期,,,,,,,,,,,,,,,
1471,2023-09-01,13935645,6878,253942232,19.10,19.10,17.80,17.80,-1.60,17.80,48,17.85,18,0.00,19.40,6.701031
1519,2023-09-01,17981005,19550,5515222823,310.00,324.00,290.00,292.50,-7.00,292.50,1,293.00,11,56.03,299.50,11.352254
1525,2023-09-01,510968,617,43359426,81.00,87.80,81.00,84.40,3.60,84.40,1,84.50,3,15.72,80.80,8.415842
1616,2023-09-01,3838766,1485,43342420,10.85,11.65,10.85,11.65,0.80,11.60,2,11.70,115,23.78,10.85,7.373272
1618,2023-09-01,5472672,2939,143184368,25.05,27.10,24.90,26.85,1.80,26.80,4,26.85,15,19.89,25.05,8.782435
1802,2023-09-01,87725325,25284,1869195460,20.05,21.65,19.90,21.65,1.95,21.65,1781,NaN,0,0.00,19.70,8.883249
1805,2023-09-01,817883,353,10299994,12.50,12.90,11.80,12.85,-0.05,12.75,17,12.85,12,0.00,12.90,8.527132
1806,2023-09-01,2302573,991,22248884,9.29,9.95,9.29,9.67,0.39,9.67,23,9.68,4,0.00,9.28,7.112069
2059,2023-09-01,1303240,2791,1163771842,915.00,927.00,872.00,874.00,-41.00,874.00,3,875.00,1,25.09,915.00,6.010929


In [7]:
#或找出股價震幅前十高的標的
df.sort_values(by =['股價震幅'], ascending= False).head(10)

,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價震幅
證券代號,日期,,,,,,,,,,,,,,,
2369,2023-09-01,30130569,12175,609178337,18.80,21.00,18.80,20.70,1.60,20.70,32,20.75,82,0.00,19.10,11.518325
1519,2023-09-01,17981005,19550,5515222823,310.00,324.00,290.00,292.50,-7.00,292.50,1,293.00,11,56.03,299.50,11.352254
2204,2023-09-01,15644037,8990,1494438829,90.00,98.70,88.60,98.70,8.90,98.70,77407,NaN,0,0.00,89.80,11.247216
2816,2023-09-01,6967795,3415,185133189,24.75,27.20,24.45,27.00,2.25,26.95,14,27.00,26,23.48,24.75,11.111111
9919,2023-09-01,14086769,3765,280454035,18.40,20.35,18.40,20.35,1.85,20.35,325,NaN,0,0.00,18.50,10.540541
6743,2023-09-01,908855,655,49459166,51.70,56.30,50.90,55.00,3.30,55.00,23,55.10,10,785.71,51.70,10.444874
3308,2023-09-01,19258315,12991,1246943166,64.40,67.90,61.80,63.50,0.50,63.40,3,63.50,14,9.42,63.00,9.682540
3588,2023-09-01,9252754,6100,608332275,61.40,67.20,61.30,67.20,6.10,67.20,10970,NaN,0,0.00,61.10,9.656301
6415,2023-09-01,6513842,6320,1969279813,288.50,312.00,284.50,305.50,18.50,305.50,6,306.00,20,47.07,287.00,9.581882


2-3
整合資料庫


In [8]:
import sqlite3

def retry_request(url, payloads, headers):
    
    for i in range(3):
    
        try:
            return requests.get(url, params= payloads, headers=headers)
        
        except:
            print('發生錯誤，請等待一分種後再嘗試')
            time.sleep(60)
    
    return None

def get_daily_price(date, connection):

    try:
        df = pd.read_sql("select * from daily_prices where 日期 = '{}'".format(date),
                         connection,
                         parse_dates= ['日期'],
                         index_col= ['證券代號','日期'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    #INDB TRUE
    #先在資料庫裡抓取每日收盤價，沒有資料則利用爬蟲重新抓取
    
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'

    payloads = {
        'response': 'html',
        'date': date.strftime('%Y%m%d'),
        'type': 'ALLBUT0999'
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'
    }

    response = retry_request(url, payloads, headers)

    try:
        df = pd.read_html(response.text)[-1]
    
    except:
        print('{}找不到資料'.format(
            date.strftime('%Y%m%d')))
        
        return None, False
    #INDB False
    df.columns = df.columns.get_level_values(2)
    df['漲跌價差'] = df['漲跌價差'].where(df['漲跌(+/-)'] != '-', -df['漲跌價差'])
    df.drop(['證券名稱','漲跌(+/-)'],inplace=True, axis=1)
    df['日期'] = pd.to_datetime(date)
    df = df.set_index(['證券代號','日期'])
    df = df.apply(pd.to_numeric, errors = 'coerce')
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
    df['股價震幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100

    return df, False

    

In [13]:
connection = sqlite3.connect('data.db')
df, in_db = get_daily_price(pd.to_datetime('2023/9/1'), connection)

print(in_db)
df

False


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價震幅
證券代號,日期,,,,,,,,,,,,,,,
0050,2023-09-01,3298119,5596,415112581,125.50,126.30,125.10,125.90,0.20,125.85,12,125.90,38,0.00,125.70,0.954654
0051,2023-09-01,57694,223,4084572,71.25,71.25,70.55,70.55,-0.35,70.50,1,70.55,47,0.00,70.90,0.987306
0052,2023-09-01,89203,222,10430606,116.80,117.30,116.65,116.85,0.00,116.65,6,116.75,21,0.00,116.85,0.556269
0053,2023-09-01,9231,1010,625518,67.90,68.00,67.50,67.50,-0.55,67.25,1,67.75,10,0.00,68.05,0.734754
0055,2023-09-01,123518,195,2894686,23.32,23.49,23.30,23.45,0.10,23.45,1,23.46,2,0.00,23.35,0.813704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2023-09-01,70188,111,1373176,19.50,19.70,19.50,19.65,0.05,19.60,2,19.65,6,5.31,19.60,1.020408
9945,2023-09-01,4446023,3364,162990865,36.85,36.90,36.50,36.60,-0.25,36.55,154,36.60,15,26.52,36.85,1.085482
9946,2023-09-01,253303,129,3621322,14.35,14.40,14.20,14.20,-0.20,14.20,10,14.25,10,33.02,14.40,1.388889


In [ ]:
import exchange_calendars as xcals
print(xcals.get_calendar_names(include_aliases=False))
tw_calendars = xcals.get_calendar("XTAI")
print(tw_calendars.opens)

In [31]:
import time
from exchange_calendars import get_calendar

def update_daily_prices(start_date, end_date, connection):

    tw_calendars = xcals.get_calendar("XTAI")

    main_df = pd.DataFrame()

    for date in pd.date_range(start_date, end_date):
        if date not in tw_calendars.opens:
            continue

        df, in_db = get_daily_price(date, connection)

        if df is not None and not in_db:
            main_df = pd.concat([main_df,df],axis = 0)
            print('{}每日收盤行情抓取完成，等待15秒'.format(date.strftime('%Y%m%d')))
            time.sleep(15)

    if not main_df.empty:
        main_df.to_sql('daily_price', connection, if_exists='append')
        return main_df

In [57]:
import time
connection = sqlite3.connect('data.db')

update_daily_prices('2024/7/29', '2024/8/5', connection)

20240729每日收盤行情抓取完成，等待15秒
20240730每日收盤行情抓取完成，等待15秒
20240731每日收盤行情抓取完成，等待15秒
20240801每日收盤行情抓取完成，等待15秒
20240802每日收盤行情抓取完成，等待15秒
20240805每日收盤行情抓取完成，等待15秒


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價震幅
證券代號,日期,,,,,,,,,,,,,,,
0050,2024-07-29,10574343,16228,1915739406,181.80,182.00,179.95,180.60,1.60,180.60,114,180.65,41,0.00,179.00,1.145251
0051,2024-07-29,183468,485,14854773,82.70,82.70,80.25,80.25,-1.20,80.20,4,80.45,1,0.00,81.45,3.007980
0052,2024-07-29,1343117,1710,237008641,177.65,177.90,175.00,175.95,1.40,175.80,30,175.95,2,0.00,174.55,1.661415
0053,2024-07-29,83453,212,7878484,97.30,97.30,93.55,94.05,-3.15,93.60,1,94.05,12,0.00,97.20,3.858025
0055,2024-07-29,569293,348,16387668,28.60,28.90,28.60,28.78,0.39,28.77,1,28.78,4,0.00,28.39,1.056710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2024-08-05,855616,485,18716230,23.20,23.20,21.30,21.60,-2.05,21.55,3,21.60,2,216.00,23.65,8.033827
9945,2024-08-05,39171183,18668,1803123023,46.90,47.20,45.00,45.35,-4.05,45.30,43,45.35,22,9.07,49.40,4.453441
9946,2024-08-05,1991054,1301,74669797,40.00,40.00,36.95,36.95,-4.10,NaN,0,36.95,289,97.24,41.05,7.429963


2-4 
篩選出當沖股票清單

In [58]:
import pandas as pd
from exchange_calendars import get_calendar
import sqlite3

def get_stock(date, connection):

    tw_calendars = xcals.get_calendar("XTAI")

    prev_trading_date = tw_calendars.previous_close(date).date()
    #previous_close: 回傳前一個交易日期

    df = pd.read_sql('SELECT * FROM daily_price WHERE 日期 = "{} 00:00:00"'.format(prev_trading_date),
                     connection, parse_dates=['日期'])
    
    codes = df[(df['收盤價'] > 5) & 
                (df['收盤價'] < 30) &
                (df['股價震幅'] > 3 ) &
                (df['成交股數'] > 50000000 )]['證券代號'].tolist()
    
    return codes
    

In [59]:
connection = sqlite3.connect('data.db')
get_stock(pd.to_datetime('2024/8/5'), connection)

['1316', '2344', '2349', '2888']

一次篩選多日股票清單

In [61]:
start = '2024/7/29'
end = '2024/8/5'
daily_target = {}

tw_calendars = xcals.get_calendar('XTAI')

for date in pd.date_range(start, end):
    
    if date not in tw_calendars.opens:
        continue

    codes = get_stock(date, connection)
    daily_target[date] = codes #dict: {date:codes}

daily_target

{Timestamp('2024-07-29 00:00:00'): [],
 Timestamp('2024-07-30 00:00:00'): ['3481'],
 Timestamp('2024-07-31 00:00:00'): ['00665L', '2349', '3481'],
 Timestamp('2024-08-01 00:00:00'): ['00637L', '2337', '2349', '2409', '3481'],
 Timestamp('2024-08-02 00:00:00'): ['2337', '2349', '2515'],
 Timestamp('2024-08-05 00:00:00'): ['1316', '2344', '2349', '2888']}